In [1]:
from utils_filter import *
from utils_datetime import *
from utils_geography import *
from utils_plotting import *
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import metpy
import numbers

In [2]:
data_location = 'data'
outlooks, pph, reports = read_datasets(data_location, 'labelled')

reading outlooks
reading pph
reading storm reports


In [16]:
# remove unneeded variables and x, y dimensions
data = pph.drop(['lat', 'lon', 'p_perfect_wind', 'p_perfect_sig_wind', 'p_perfect_hail', 'p_perfect_sig_hail', 'p_perfect_tor', 'p_perfect_sig_tor', 'RAMP_CAT', 'p_perfect_total', 'PPH_CAT', 'REPORT_NUM', 'WINDSP_CAT', 'HAILSZ_CAT'])

# make data numerical
data
# normalize

<xarray.Dataset>
Dimensions:     (time: 16071, x: 93, y: 65)
Coordinates:
  * time        (time) object '197901010000' '197901020000' ... '202212310000'
  * x           (x) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 88.0 89.0 90.0 91.0 92.0
  * y           (y) float64 0.0 1.0 2.0 3.0 4.0 5.0 ... 60.0 61.0 62.0 63.0 64.0
Data variables: (12/15)
    MAX_CAT     (time) object ...
    RAMP_UP     (time) int32 ...
    RAMP_DOWN   (time) int32 ...
    SEASON      (time) object ...
    REGION      (time) object ...
    PPH_NUM     (time) float64 ...
    ...          ...
    TOR_F       (time) object ...
    WINDSP_NUM  (time) float64 ...
    HAILSZ_NUM  (time) float64 ...
    NEIGH_NUM   (time) float64 ...
    RMSE_NUM    (time) float64 ...
    BS_NUM      (time) float64 ...
Attributes:
    title:         Practically Perfect Wind Hindcasts
    grid:          80-km NCEP 211
    sigma:         1.5
    author:        Dr. Victor Gensini
    author_email:  vgensini@niu.edu
    citation:      https://doi.org/10.1175/BAMS-D-19-0321.1

In [ ]:
# do pca

In [ ]:
# cluster with, k-means, knn, look here for use cases https://scikit-learn.org/stable/modules/clustering.html